In [1]:

import numpy as np
import pandas as pd
from pathlib import Path
import gc
import itertools
import importlib
from IPython.display import clear_output

# BOKEH
import bokeh.plotting as bk
import bokeh.models as bkmod
import bokeh.layouts as bklay
import bokeh.palettes as bkpalettes
import bokeh.util.hex as bkhex
import bokeh.transform as bktrfm
import bokeh.colors as bkcolors


# xsuite
import xtrack as xt
import xmask as xm
import xfields as xf
import xpart as xp

# BBStudies
import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.XMask.Utils as xutils
import BBStudies.Physics.Constants as cst
import BBStudies.Plotting.Bokeh.Tools as bktools
import BBStudies.Plotting.Bokeh.Presets as bkpresets
import BBStudies.Physics.Base as phys

bk.output_notebook()
def plot_phase_space(df,rfbucket = None):
    _FIGS   = {}
    _default_fig_width  = 1800
    _bot_tab_height     = 400
    padding             = 20 
    # Phase space plots:
    #=====================================
    adjustment = 0
    _FIGS['x-y']  = bkpresets.make_scatter_fig(df,xy=('x_sig','y_sig')      ,alpha=0.2,title=f'x-y plane ({len(df)} particles)' ,width=int(_default_fig_width/4.5)+adjustment,height=_bot_tab_height,padding=padding)
    _FIGS['x-px'] = bkpresets.make_scatter_fig(df,xy=('x_sig','px_sig')     ,alpha=0.2,title=f'x norm. phase space'             ,width=int(_default_fig_width/4.5),height=_bot_tab_height,padding=padding)
    _FIGS['y-py'] = bkpresets.make_scatter_fig(df,xy=('y_sig','py_sig')     ,alpha=0.2,title='y norm. phase space'              ,width=int(_default_fig_width/4.5),height=_bot_tab_height,padding=padding)
    _FIGS['zeta-pzeta'] = bkpresets.make_scatter_fig(df,xy=('zeta','pzeta') ,alpha=0.2,title='zeta phase space'                 ,width=int(_default_fig_width/4.5),height=_bot_tab_height,padding=padding)


    _FIGS['x-px'].min_border_left  = padding
    bktools.set_aspect(_FIGS['x-y']         , x_lim=(-2,11),y_lim=(-2,11), aspect=1, margin=padding-adjustment)
    bktools.set_aspect(_FIGS['x-px']        , x_lim=(-11,11),y_lim=(-11,11), aspect=1, margin=0)
    bktools.set_aspect(_FIGS['y-py']        , x_lim=(-11,11),y_lim=(-11,11), aspect=1, margin=0)
    # bktools.set_aspect(_FIGS['zeta-pzeta'] , x_lim=(-1,1),y_lim=(-1,1), aspect=1, margin=0)
    _FIGS['zeta-pzeta'].x_range  = bkmod.Range1d(-0.5, 0.5)
    _FIGS['zeta-pzeta'].y_range  = bkmod.Range1d(-5e-4, 5e-4)

    _FIGS['x-y'].xaxis.axis_label = r'$$\tilde x /\sqrt{\varepsilon_{x}}$$'
    _FIGS['x-y'].yaxis.axis_label = r'$$\tilde y /\sqrt{\varepsilon_{y}}$$'

    _FIGS['x-px'].xaxis.axis_label = r'$$\tilde x /\sqrt{\varepsilon_{x}}$$'
    _FIGS['x-px'].yaxis.axis_label = r'$$\tilde p_x /\sqrt{\varepsilon_{x}}$$'

    _FIGS['y-py'].xaxis.axis_label = r'$$\tilde y /\sqrt{\varepsilon_{y}}$$'
    _FIGS['y-py'].yaxis.axis_label = r'$$\tilde p_y /\sqrt{\varepsilon_{y}}$$'

    _FIGS['zeta-pzeta'].xaxis.axis_label = r'$$\zeta$$'
    _FIGS['zeta-pzeta'].yaxis.axis_label = r'$$p_\zeta $$'

    if rfbucket is not None:
        color = 'lightseagreen'
        ls    = 'solid'
        label = f'RF Bucket'
        for zcut in list(np.linspace(0.001,rfbucket.zeta_max,10)) + [0.09,2*0.09,3*0.09]:
            zeta_vec,delta_vec = rfbucket.invariant(zcut,npoints = 1000)

            if zcut/0.09 in [1,2,3]:
                color = 'mediumvioletred'
                ls    = 'dotted'
                label = f'σ'
            
            line_top = _FIGS['zeta-pzeta'].line(x=zeta_vec,y=delta_vec, line_width=2, color=color, alpha=0.2, line_dash=ls, legend_label=label)
            line_bot = _FIGS['zeta-pzeta'].line(x=zeta_vec,y=-delta_vec, line_width=2, color=color, alpha=0.2, line_dash=ls, legend_label=label)
            line_top.level = 'underlay'
            line_bot.level = 'underlay'
            # plt.plot(zeta_vec,delta_vec,'-',color='C2')
            # plt.plot(zeta_vec,-delta_vec,'-',color='C2')
    


    grid = bklay.gridplot([[_FIGS['x-y'],_FIGS['x-px'] ,_FIGS['y-py'] ,_FIGS['zeta-pzeta']]],toolbar_location='right')
    #=====================================
    return grid

def phase_space_labels(fig,axes):
    import matplotlib.pyplot as plt
    # Adding labels
    for plane,ax in zip(['x','y','zeta'],axes):
        plt.sca(ax)
        plt.axis('equal')
        if plane == 'zeta':
            plane = '\zeta'
        plt.xlabel(rf'$\tilde {plane}/\sqrt{{\varepsilon_{plane}}}$')
        plt.ylabel(rf'$\tilde p_{plane}/\sqrt{{\varepsilon_{plane}}}$')
        
        if plane == '\zeta':
            plt.xlim(-0.5,0.5)
            plt.ylim(-0.5,0.5)
        else:
            plt.xlim(-6,6)
            plt.ylim(-6,6)


Loading BokehJS ...

In [2]:
import importlib
importlib.reload(bkpresets)

<module 'BBStudies.Plotting.Bokeh.Presets' from '/storage-hpc/gpfs_data/HPC/home_recovery/phbelang/abp/BBStudies/BBStudies/Plotting/Bokeh/Presets.py'>

---
# Loading collider for RFBucket
---

In [2]:
collider_path   = 'colliders/collider_BUNCH_0000.json'
collider        = xt.Multiline.from_json(collider_path)
line            = collider['lhcb1']
line.build_tracker()
rfbucket        = xPlus.RFBucket(line)
clear_output()

---
# Generating particles
---

In [9]:



n_part = 2400

delta_theta = np.pi/2/50
x,y = phys.serpentine_path( r_min       = 0.25,
                            r_max       = 10,
                            num_steps   = 40,
                            theta_min   = delta_theta,
                            theta_max   = np.pi/2-delta_theta,
                            num_points  = 5000,
                            smooth_factor=0.2)


x,y = phys.resample(x,y,n_part)


# Initializing:
#======================================================
coord_df = pd.DataFrame({   'x_sig' :np.zeros(n_part),
                            'px_sig':np.zeros(n_part),
                            'y_sig' :np.zeros(n_part),
                            'py_sig':np.zeros(n_part),
                            'zeta'  :np.zeros(n_part),
                            'delta' :np.zeros(n_part)})
#======================================================


# Overwriting
coord_df.loc[:,'x_sig']  = x
coord_df.loc[:,'y_sig']  = y
coord_df.loc[:,'zeta']   = 3*0.09


# Plotting
coord_df.insert(5,'pzeta',xPlus.delta2pzeta(coord_df['delta'],beta0=1))
coord_df.insert(0,'particle',coord_df.index)
particle_id = coord_df.index
bk.show(plot_phase_space(coord_df,rfbucket = rfbucket))
display(coord_df)

/storage-hpc/gpfs_data/HPC/home_recovery/phbelang/abp/BBStudies/BBStudies/Physics/Base.py:44: RuntimeWarning: overflow encountered in exp
  return step_height / (1 + np.exp(-2 * (x - step_location) / step_width))
/tmp/ipykernel_17841/183742867.py:89: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  grid = bklay.gridplot([[_FIGS['x-y'],_FIGS['x-px'] ,_FIGS['y-py'] ,_FIGS['zeta-pzeta']]],toolbar_location='right')
/tmp/ipykernel_17841/183742867.py:89: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  grid = bklay.gridplot([[_FIGS['x-y'],_FIGS['x-px'] ,_FIGS['y-py'] ,_FIGS['zeta-pzeta']]],toolbar_location='right')


,particle,x_sig,px_sig,y_sig,py_sig,zeta,pzeta,delta
0,0,0.007853,0.0,0.249877,0.0,0.27,0.0,0.0
1,1,0.132225,0.0,0.212162,0.0,0.27,0.0,0.0
2,2,0.220862,0.0,0.117114,0.0,0.27,0.0,0.0
3,3,0.275724,0.0,0.010612,0.0,0.27,0.0,0.0
4,4,0.407156,0.0,0.012993,0.0,0.27,0.0,0.0
...,...,...,...,...,...,...,...,...
2395,2395,0.839075,0.0,9.964723,0.0,0.27,0.0,0.0
2396,2396,0.707994,0.0,9.974893,0.0,0.27,0.0,0.0
2397,2397,0.576790,0.0,9.983321,0.0,0.27,0.0,0.0
2398,2398,0.445487,0.0,9.990057,0.0,0.27,0.0,0.0


In [12]:
np.sqrt(x**2+y**2)

array([ 0.25      ,  0.24997935,  0.24998931, ...,  9.99992915,
        9.99999256, 10.        ])

---
# OLD
---

In [ ]:
# density     = 8/(2*np.pi*1)
# _generators = [pd.DataFrame({'x_sig':[0],'y_sig':[0]})]
# delta = 0
# for r in np.linspace(0,10,30)[1:]:
#     n_points = int(2*np.pi*r*density)
#     _df  = phys.polar_grid( r_sig     = [r],
#                                 theta_sig = np.linspace(0 + delta,np.pi/2 - delta,n_points))
    
#     _generators.append(_df)


# generator = pd.concat(_generators).reset_index()
# n_part = len(generator)

---
# Saving Results
---

In [10]:
coord_df.to_parquet('particles/SERPENT_XY_ZETA_3.parquet')

---
# Load and plot
---

In [ ]:

coord_df = xPlus.import_parquet('particles/SERPENT_XY_ZETA_0.parquet')
bk.show(plot_phase_space(coord_df,rfbucket = None))
display(coord_df)